In [1]:
import warnings
warnings.filterwarnings('ignore')
import xml.etree.ElementTree as ET
import html.parser
from xml.sax.saxutils import unescape
import nltk
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.chunk import ne_chunk
from nltk.tag import pos_tag
from nltk.tree import Tree
import string
from nltk.util import bigrams,everygrams
import csv
import re

In [2]:
doc = open('CutData/Pseudo/NTF_200/Features_NTF2_1/Doc_NTF2_1.csv', 'r').read() #str
doc_num = doc.split('\n')  # str to list,以','切割

In [3]:
tree = ET.parse('Formspring/new_XMLMergedFile.xml')
root = tree.getroot()
symbol = {'!', '"', '#', '$', '%', '&',"'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', 
          '=', '>', '?', '@', '[',  ']', '^', '_', '`', '{', '|', '}', '~', '\''}
html_parser = html.parser.HTMLParser()
clean_txt = []
c = []
for child in root.iter(tag='TEXT'):
    if child.attrib["id"] in doc_num:
        # 去除Q/A
        find_q = html_parser.unescape(child.text)[0:3]    # 找前2個字母Q:
        remove_q = html_parser.unescape(child.text).replace(find_q, '')    # 移除Q:
        find_a = html_parser.unescape(child.text).split('<br>')[1][0:3]    # 1.由<br>切割後, 找第2部分的前2個字母A:
        remove_a = remove_q.replace(find_a, '')    #移除A:
        txt = remove_a.split('<br>') # 切Q/A
        a =[]

        for i in range(0,2):
            sent = sent_tokenize(txt[i]) # list,斷句
            for h in range(0,len(sent)):
                re_url = re.sub(r'http://[a-zA-Z0-9./?%&_=-]*','' ,sent[h])
                for x in symbol:
                    if x in re_url:
                        re_url = re_url.replace(x, '').strip()    #去除符號
                word = word_tokenize(re_url)    # 斷詞
    #             print(word)
    #             big = list(bigrams(word))    # 2-ngram
                a.extend(word)
        b = list(set(a))    # 去除重複
        b.sort(key=a.index)    # 排序
    #     print(b)    
    #     print('-----')
        c.extend(b)

In [4]:
d = list(set(c))     # 去除重複
d.sort(key=c.index)    # 排序
# print(d)# 全域

In [5]:
tree = ET.parse('Formspring/new_XMLMergedFile.xml')
root = tree.getroot()
symbol = {'!', '"', '#', '$', '%', '&',"'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', 
          '=', '>', '?', '@', '[',  ']', '^', '_', '`', '{', '|', '}', '~', '\''}
html_parser = html.parser.HTMLParser()
clean_txt = []
j = 0
exl = []
for child in root.iter(tag='TEXT'):
    if child.attrib["id"] in doc_num:
        # 去除Q/A
        find_q = html_parser.unescape(child.text)[0:3]    # 找前2個字母Q:
        remove_q = html_parser.unescape(child.text).replace(find_q, '')    # 移除Q:
        find_a = html_parser.unescape(child.text).split('<br>')[1][0:3]    # 1.由<br>切割後, 找第2部分的前2個字母A:
        remove_a = remove_q.replace(find_a, '')    #移除A:
        txt = remove_a.split('<br>') # 切Q/A
        a = []
        for i in range(0,2):
            sent = sent_tokenize(txt[i]) # list,斷句
            for h in range(0,len(sent)):
                re_url = re.sub(r'http://[a-zA-Z0-9./?%&_=-]*','' ,sent[h])
                for x in symbol:
                    if x in re_url:
                        re_url = re_url.replace(x, '').strip()    #去除符號
                word = word_tokenize(re_url)    # 斷詞
    #             big = list(bigrams(word))    # 2-ngram
                a.extend(word)

        e = []
        for q in d:
            count = 0
            for w in a:
                if q == w:
    #                 print(q)
                    count+=1
            e.append(count)
    #         print(count)
        j += 1
    #     print(e)
        exl.append([])
        exl[j-1].append("D" + str(child.attrib["id"]))
        for jj in e:
            exl[j-1].append(jj)
    #     print('------------') 

In [6]:
# print(d)

In [7]:
print(len(d))

2296


In [8]:
d.insert(0,'Doc') #120681組25702

In [9]:
#計算小於2的改為空直
for col in range(1,len(d)):
    count = 0
    for row in range(0,len(exl)):
        count+=exl[row][col]
#     print(count)
    if (count<2):
        for chg in range(0,len(exl)):
            exl[chg][col] = '' #欄位變空值
            d[col] = ''  #uni變空值
# print(listbig)

In [10]:
#不為空的uni放入list
d = [remv for remv in d if remv !='']

In [11]:
print(len(d))

730


In [12]:
# print(d) #4287

In [13]:
#不為空的欄位放入list
new = []
for g in range(0,len(exl)):
    new.append([])
    for k in exl[g]:
#         print(k)
#         print('--------')
        if k != '':
#             print(k)
            new[g].append(k)
# print(new[0])

In [14]:
print(len(new[0]))

730


In [15]:
print(len(new))

400


In [16]:
with open('CutData/Pseudo/NTF_200/Features_NTF2_1/Unigram_NTF2_1.csv', 'w', encoding='UTF-8', newline='') as csvfile:
  # 以空白分隔欄位，建立 CSV 檔寫入器
    writer = csv.writer(csvfile)
    writer.writerow(d)
    writer.writerows(new)